# Pre-procesing 

In [453]:
from random import choice
from collections import defaultdict
from datetime import datetime
from pathlib import Path
from shutil import rmtree
import pandas as pd
import json
import dateutil.parser
import os

In [454]:
#Datos de kaggle
RAW_DATA = Path("raw_dataset").absolute()

#Output de datos (datos limpios)

CLEAN_DATA = Path("clean_dataset").absolute()

if  CLEAN_DATA.exists():
    rmtree(CLEAN_DATA)
    os.mkdir(CLEAN_DATA)
else:
    os.mkdir(CLEAN_DATA)

## Loading the data

In [455]:

countries = ["CA","GB","US"] #Lista de paiess.
videos = {} #Lista con los dataframes de archivos csv
categories = {} # Lista con los diccionarios de los archivos json.
print("Abriendo archivos del dataset-->")
for i,country in enumerate(countries):
    print(f"\tPais: {country}  {i+1}/{len(countries)} ",end = "\r")
    file_csv = RAW_DATA.joinpath(f"{country}videos.csv")
    videos[country] = pd.read_csv(file_csv,encoding="ISO-8859-1",lineterminator="\n")
    videos[country].columns = [x.strip() for x in videos[country].columns]


    with open(RAW_DATA.joinpath(f"{country}_category_id.json")) as file:
        items = json.load(file)["items"]
        temp = {int(x["id"]): x for x in items} #La llave de cada categoria es su id.
        val = defaultdict(lambda: "NULL")
        val["snippet"] = defaultdict(lambda:"NULL")
        val["snippet"]["title"] = 'No categoria'
        categories[country] = defaultdict(lambda : val,temp)
print("\nDatos Cargados!")

Abriendo archivos del dataset-->
	Pais: US  3/3 
Datos Cargados!


In [456]:
# veamos el numero de nulos sin contar la descripcion de cada DF
print("Numero de filas con algun nulo (sin contar descripcion) en el DF de ")
for key,df_clean in videos.items():
    df = df_clean.copy().drop(["description"],axis = 1)
    df1 = df[df.isna().any(axis=1)]
    print(f"\tPais --> {key}: {len(df1)}")


Numero de filas con algun nulo (sin contar descripcion) en el DF de 
	Pais --> CA: 0
	Pais --> GB: 0
	Pais --> US: 0


In [457]:
def show_random_head(): #Funcion para mostrar el head de alguno de los df's
    pick = choice(countries)
    print("Mostrando head de {}".format(pick))
    return videos[pick].head

In [458]:
show_random_head()() #Mostramos uno de los df

Mostrando head de GB


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: â¦Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs â¦Ready f...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


In [459]:
def get_category_name(series,key):
    return series.apply(lambda x: categories[key][x]["snippet"]["title"])

def get_publish_timestamp(series):
    return series.apply(lambda x: dateutil.parser.isoparse(x).timestamp())

def get_trending_timestamp(series):
    return series.apply(lambda x: datetime.strptime(x,"%y.%d.%m").timestamp())

In [460]:
for key,df in videos.items():
    df["category"] = get_category_name(df["category_id"],key) # Agregamos el nombre de la categoria
    # Parseamos las fechas para llegar y comparar como timestamps
    df["publish_timestamp"] = get_publish_timestamp(df["publish_time"]) 
    df["trending_timestamp"] = get_trending_timestamp(df["trending_date"]) 
    df.drop(["video_id","category_id","thumbnail_link"],axis =1,inplace=True)

In [461]:
print("Escribiendo nuevo dataset-->")
df_all = pd.DataFrame(columns = ["country"]+ list(videos["CA"].columns))
for i,(key,df) in enumerate(videos.items()):
    df["country"] = key
    print(f"Escribiendo {key}videos.csv   {i+1}/{len(videos.items())}",end = "\r")
    # get 10% sample of df
    # remove rows with no tags
    df = df[df["tags"] != "[none]"]
    df = df.sample(frac=0.1)
    df_all = pd.concat([df_all,df],ignore_index = True)
# Add numeric id to each row called video_id
df_all["video_id"] = df_all.index

print("\n Dataset listo!")

Escribiendo nuevo dataset-->
Escribiendo USvideos.csv   3/3
 Dataset listo!


In [462]:
df_all.head()

,country,trending_date,title,channel_title,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,category,publish_timestamp,trending_timestamp,video_id
0,CA,17.14.11,HOLIDAY 2017 â­ï¸ COLLECTION & SKIN FROST PA...,jeffreestar,2017-11-12T17:51:45.000Z,"jeffree star|""jeffree star cosmetics""|""jeffree...",1637172,100779,989,18966,False,False,False,HEY EVERYONE!!!! â­ï¸ Today I'm revealing my...,Howto & Style,1.510509e+09,1.510628e+09,0
1,CA,17.24.12,Dude Perfect Face Off | What's In The Box,Whistle Sports,2017-12-20T23:00:00.000Z,"dude perfect|""dude perfect christmas""|""dude pe...",2353300,35429,868,2162,False,False,False,This Twin Battle is sponsored by Dollar Shave ...,Sports,1.513811e+09,1.514084e+09,1
2,CA,18.20.03,Pakistani Drama | Lamhay - Episode 2 | Aplus D...,Best Pakistani Dramas,2018-03-19T16:57:50.000Z,"lamhay|""aplus""|""Aplus""|""APlus""|""Aplus new dram...",45420,346,38,104,False,False,False,Pakistani Drama | Lamhay - Episode 2 | Aplus D...,Entertainment,1.521479e+09,1.521515e+09,2
3,CA,18.11.02,Casually Explained: Introverts and Extraverts,Casually Explained,2018-02-09T14:00:04.000Z,"introverts and extraverts|""introverts""|""extrav...",1149976,72886,798,4024,False,False,False,And he makes the big return on his bday!\nFair...,Comedy,1.518185e+09,1.518318e+09,3
4,CA,17.25.12,SZA - The Weekend (Official Video),SZAVEVO,2017-12-22T05:00:01.000Z,"R&B|""SZA""|""The Weekend""|""Top Dawg Entertainmen...",2623483,109167,12985,16651,False,False,False,SZA's CTRL available on:\nApple Music - http:/...,Music,1.513919e+09,1.514171e+09,4


# New Tables for tags

In [463]:
tags_series = df_all["tags"].apply(lambda x: x.split("|"))
tags_series = tags_series.apply(lambda x: [y.strip() for y in x])
tags = [item for sublist in tags_series for item in sublist] # Lista con todas las tags.
new_tags = []
for tag in tags:
    # remove " from tags
    tag = tag.replace('"',"")
    # Replace spaces with _
    tag = tag.replace(" ","_")
    new_tags.append(tag)

tags = new_tags


# Count frequency of each tag
freq_dict = {}
for tag in tags:
    if tag in freq_dict:
        freq_dict[tag] += 1
    else:
        freq_dict[tag] = 1

# Remove tags that appear less than 100 times

thresh = 100

freq_dict = {key:value for key,value in freq_dict.items() if value >= thresh}
tag_set = freq_dict.keys()


# Remove from every elemnt in tag series values that are not in final_set
def remove_tags(x):
    res = []
    for elem in x:
        elem  = elem.replace('"',"")
        # Replace spaces with _
        elem = elem.replace(" ","_")
        if elem in tag_set:
            res.append(elem)

    return res

tags_series = tags_series.apply(remove_tags)



# print info

print("\n------------BEFORE------------\n")
print(f"Number of unique tags : {len(set(new_tags))}")
print(f"Total number of tags: {len(new_tags)}")
print("\n------------AFTER------------\n")

print(f"Number of unique tags : {len(tag_set)}")
print("Total number of tags.",sum(freq_dict.values()))
print("Rows without tags {}".format(len(tags_series[tags_series.apply(lambda x: len(x) == 0)])))



------------BEFORE------------

Number of unique tags : 71212
Total number of tags: 229598

------------AFTER------------

Number of unique tags : 93
Total number of tags. 17166
Rows without tags 5372


In [464]:
# create new dataframe with 2 columns tag_id and tag_name
tag_df = pd.DataFrame(columns = ["tag_id","tag_name"],index=  range(len(tag_set)))
# create enw dataframe with 2 columns video_id and tag_id
video_tag_df = pd.DataFrame(columns = ["video_id","tag_id"],index = range(sum(freq_dict.values())))

In [465]:
id_dict = {}
for i,elem in enumerate(tag_set):
    tag_df.loc[i] = [i,elem]
    id_dict[elem] = i
counter = 0
for elem in enumerate(tags_series):
    L = elem[1]
    for tag in L:
        video_tag_df.loc[counter] = [elem[0],id_dict[tag]]
        counter += 1


In [466]:
video_tag_df

,video_id,tag_id
0,3,0
1,3,1
2,5,2
3,12,3
4,12,4
...,...,...
17161,11478,54
17162,11478,71
17163,11479,92
17164,11481,16


In [467]:
tag_df

,tag_id,tag_name
0,0,comedy
1,1,animation
2,2,funny_video
3,3,food
4,4,cooking
...,...,...
88,88,pop
89,89,cute
90,90,star_wars
91,91,music_video


In [468]:
df_all = df_all.drop(["tags"],axis = 1)


## Saving the new dataset

In [469]:
# Save dataframes to csv
df_all.to_csv(CLEAN_DATA /"videos.csv",index = False)
tag_df.to_csv(CLEAN_DATA /"tags.csv",index = False)
video_tag_df.to_csv(CLEAN_DATA /"video_tags.csv",index = False)